In [1]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch
from tqdm.notebook import tqdm, trange
import gc

In [2]:
# MODEL_NAME = "meta-llama/Llama-2-7b-hf"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
model = model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.33.2"
}



In [3]:
def generate_response(prompt: str, max_new_tokens: int = 128, temperature: float = 0.001) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.001,
            generation_config=generation_config,

            do_sample = True,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)


# def format_prompt(prompt, history='', Background=''):
#     return f"""<s>[INST] <<SYS>>
# {{ 
#     {Background}
#     {history}
# }}
# <</SYS>>
# [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST]  [/INST]  </s><s>[INST] {{ {prompt} }} [/INST]"""

def format_prompt(prompt, history='', Background=''):
    return f"""<s>[INST] <<SYS>>
{{ 
    {Background}
    {history}
}}
<</SYS>>
[/INST]  </s>
<s>[INST] {{ {prompt} }} [/INST]"""


# def format_prompt(prompt, history='', Background=''):
#     return f"""
#     {Background}
#     {history}
#     {prompt}"""

In [4]:
The_Topic = '"ways to detect if a news is fake or real"'
User_A = "Tom"
User_B = "Jerry"
 
Background_for_A = f'You are {User_A}. The topic of the discussion id {The_Topic}. Ask a question or write a response that answers the question of {User_B} according to the conversation history, or with a new information. Answer in one short sentence only and strictly, do not repeat what is already said in the conversation. If you have nothing new to add, reply with a thanks.'
Background_for_B = f'You are {User_B}. The topic of the discussion id {The_Topic}. Ask a question or write a response that answers the question of {User_A} according to the conversation history, or with a new information. Answer in one short sentence only and strictly, do not repeat what is already said in the conversation. If you have nothing new to add, reply with a thanks.'
Background = ''
the_output = ''

history = f'''
{User_A}: Hello, I am {User_A}
{User_B}:  Hello {User_A}, I am {User_B}
'''
prompt_to_A = f"Hello {User_A}, I am {User_B}"

for j in trange(1,30):
    try:
        prompt_to_B = generate_response(format_prompt(prompt_to_A, history, Background_for_A), 150, 0.001)
        if prompt_to_B[:len(User_A)] == User_A:
            history += prompt_to_B + '\n'
        else:
            history += f"{User_A}: " + prompt_to_B + '\n'

        prompt_to_A = generate_response(format_prompt(prompt_to_B, history, Background_for_B), 150, 0.001)
        if prompt_to_A[:len(User_B)] == User_B:
            history += prompt_to_A + '\n'
        else:
            history += f"{User_B}: " + prompt_to_A + '\n'
        
        if not j % 5:
            summary = generate_response(format_prompt("What are the keypoints of above chat and summary, in very very short.", history), 500)
            summary = '\n'.join(summary.strip().split('\n')[1:])
            history = "We know that\n" + summary +'\n' + f'''
{User_A}: Hello, I am {User_A}
{User_B}:  Hello {User_A}, I am {User_B}
'''
    except:
        break




SyntaxError: incomplete input (3329911007.py, line 40)

In [ ]:
print(history)

We know that
* Common signs of a fake news article include lack of credible sources, sensationalized headlines, and emotional appeals.
* Clickbait headlines and overly simplistic or sensationalized language can also indicate a fake news article.
* Unverifiable or unreliable sources, such as anonymous sources or sources with a clear bias or agenda, can be a sign of a fake news article.
* Manipulated or altered images, videos, or other media can be easily debunked with a simple fact-check.
* It's important to fact-check visual evidence presented in an article to ensure its authenticity.
* Being vigilant and skeptical when consuming news is crucial in today's digital age where misinformation can spread quickly.

Tom: Hello, I am Tom
Jerry:  Hello Tom, I am Jerry
 Thanks, Jerry! That's a great point. It's important to fact-check not only the text of a news article but also any visual evidence or images included in it. This can help us identify manipulated or altered images that may be used

In [ ]:
summary = generate_response(format_prompt("What are the keypoints of above chat and summary, in very very short.", history), 500)
newsummary = '\n'.join(summary.strip().split('\n')[1:])

print(summary)

 Sure, here are the key points of the chat in very short summary:
* Common signs of a fake news article include lack of credible sources, sensationalized headlines, and emotional appeals.
* Fake news articles often use clickbait headlines, overly simplistic or sensationalized language, and unverifiable sources.
* Lack of fact-checking or evidence to support claims is also a red flag.
* Manipulated or altered images or videos can be used to support false claims.
* Emotional appeals and loaded language can be used to manipulate the reader's emotions rather than presenting balanced and factual information.


In [ ]:
# newsummary = '\n'.join(summary.strip().split('\n')[1:])
# print(newsummary)
print(len(history))
print(len(summary))

6608
704


In [ ]:
# Define the string you want to append to the file
text_to_append = "This is additional text to append to the file."

# Specify the file path where you want to append the text
file_path = f"./{The_Topic}.txt"
file_path = f"./AGI_tobeinvented.txt"

# Open the file in append mode ('a' for appending)
with open(file_path, 'a') as file:
    # Write the string to the file
    file.write("Conversation\n\n")
    file.write(history)
    file.write("Summary\n\n")
    file.write(summary)

# Close the file when you're done (the 'with' statement automatically does this)


NameError: name 'summary' is not defined

In [ ]:
# history_for_A = '''
# User A: Hello, I am User A. 
# User B:  Hello User A, I am User B.
# '''

# history_for_B = '''
# User A: Hello, I am User A.
# User B: Hello User A, I am User B.
# '''

In [ ]:
# Fixed_Prefix = "Write a response that answers the request according to the conversation history below. Answer in one short sentence only."
# Background_for_A = 'Talk about Fake news with User B'
# Background_for_B = 'Talk about Fake news with User A'
# Background = ''
# the_output = ''
# Fixed_Prefix_len = len(Fixed_Prefix)
# import sys
# history = '''
# User A: Hello, I am User A.
# User B:  Hello!
# '''
# for j in range(1,50):
#     if not j % 5:    
#         summary = generate_response(format_prompt("What are the keypoints of above chat and summary, in very very short.", history), 500)
#         Background += '\n' + ''.join(Background.strip().split('\n')[1:])
#         history = '\n'
#     print(Background)
#     # print(history)
#     print(the_output)
#     print(j, "*" * 50)
#     while True:
#         prompt = input()
#         if len(prompt) == 0:
#             continue
#         else:
#             break
#     if prompt == 'quit':
#         break
#     the_output = generate_response(format_prompt(prompt, history), 200, 0.001)
#     history += "Request: " + prompt + '\n'
#     history += "Response: " + the_output + '\n\n'